In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(solution):
    sum_gain = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        gain = res[1]-res[2]
        sum_gain += gain
    return (sum_gain,)

"""
def fitness_function(solution):
    defeated = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        if res[2]<=0:
            defeated += 1
    return (defeated,)
"""

'\ndef fitness_function(solution):\n    defeated = 0\n    for enemy in range(1,9):\n        env.update_parameter("enemies",[enemy])\n        res = env.play(np.array(solution))\n        if res[2]<=0:\n            defeated += 1\n    return (defeated,)\n'

In [4]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [5]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [6]:
def best_initializer(pop_size,in_sigma,in_mutpb,in_tau):
    pop = np.random.choice(final_results['solution'],pop_size)
    pop = [creator.Individual(ind) for ind in pop]
    for i in range(len(pop)):
        if np.random.uniform() < in_mutpb:
            pop[i] = mutGaussianAdaptive(pop[i],in_sigma,in_tau,1)
    return pop

In [7]:
def genetic_algorithm(in_sigma,in_tau,in_mutpb,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats
    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-0,0)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = best_initializer(pop_size,in_sigma,in_mutpb,in_tau)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,pop)):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,invalid_ind)):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)
        pool.close()
        pool.join()

    return hof,logbook

In [8]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [9]:
def objective(trial):
    
    params = {'in_sigma':trial.suggest_float("in_sigma", 0.1, 1., step=0.1),
              'in_tau':trial.suggest_float("in_tau", 0.1, 1., step=0.1),
              'in_mutpb':trial.suggest_float("in_mutpb", 0.1, 1., step=0.1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(1):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['max_hof'].max()


In [12]:
import pickle
with open('/kaggle/input/tune-res-generalist/final_solutions_v2.pickle', 'rb') as handle:
    final_results = pickle.load(handle)

In [49]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-15 17:04:56,718] A new study created in memory with name: GA


gen	nevals	Mean    	Max	Std    	max_hof
0  	50    	-195.456	200	309.717	200    
1  	186   	54.824  	180	85.6383	200    
2  	185   	92.444  	180	54.6836	200    
3  	187   	123.868 	199.6	56.7428	200    
4  	192   	137.236 	214.6	51.1156	214.6  
5  	184   	148.012 	212.8	47.9017	214.6  
6  	184   	160.352 	240.8	52.3193	240.8  
7  	181   	179.636 	263.8	44.8513	263.8  
8  	188   	190.352 	263.8	51.5401	263.8  
9  	186   	203.552 	229  	34.936 	263.8  
10 	182   	188.884 	233  	50.3883	263.8  
11 	190   	187.332 	233  	55.1897	263.8  
12 	188   	197.796 	266.6	46.7111	266.6  
13 	189   	196.776 	266.6	45.4992	266.6  
14 	181   	204.548 	240  	34.6914	266.6  
15 	187   	190.304 	226.4	39.7086	266.6  
16 	185   	193.624 	231.8	45.8103	266.6  
17 	187   	204.38  	243.6	34.9299	266.6  
18 	186   	193.228 	252.8	42.3189	266.6  
19 	186   	197.36  	227.2	39.6446	266.6  
20 	190   	195.56  	245  	44.4498	266.6  
21 	191   	198.764 	249.8	47.2127	266.6  
22 	185   	199.236 	249.8	38.0056	266.6  


[I 2023-10-15 17:17:53,568] Trial 0 finished with value: 356.8000000000009 and parameters: {'in_sigma': 0.7000000000000001, 'in_tau': 0.9, 'in_mutpb': 0.6, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 1, 'cxpb': 0.30000000000000004, 'cx_type': 'whole_aritmetic', 'alpha': 'random', 'mutpb': 0.9, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 0.1, 'tau': 0.8, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 356.8000000000009.


30 	186   	213.148 	291.8	54.8213	356.8  
gen	nevals	Mean   	Max	Std    	max_hof
0  	50    	115.436	249	131.432	249    
1  	150   	-214.264	200	148.036	249    
2  	148   	-162.688	200	147.734	249    
3  	147   	-55.128 	213.2	116.013	249    
4  	149   	-6.396  	213.2	106.086	249    
5  	149   	31.32   	213.2	83.0505	249    
6  	150   	21.9    	165.8	81.0058	249    
7  	150   	23.992  	149  	90.903 	249    
8  	148   	30.268  	149  	49.6006	249    
9  	149   	35.932  	156.2	98.4415	249    
10 	149   	34.268  	208.6	80.413 	249    
11 	148   	87.044  	259.8	105.43 	259.8  
12 	146   	92.704  	259.8	100.169	259.8  
13 	148   	94.648  	259.8	94.5954	259.8  
14 	150   	37.404  	259.8	75.7585	259.8  
15 	150   	45.292  	187.6	79.2782	259.8  
16 	147   	49.244  	231.4	90.2203	259.8  
17 	149   	13.82   	165.6	72.4471	259.8  
18 	150   	18.948  	174.2	70.6648	259.8  
19 	150   	-7.04   	227.4	107.761	259.8  
20 	146   	26.272  	280.4	96.7469	280.4  
21 	146   	35.724  	182  	92.9361	280.4  
22

[I 2023-10-15 17:28:13,398] Trial 1 finished with value: 301.8000000000007 and parameters: {'in_sigma': 0.30000000000000004, 'in_tau': 0.7000000000000001, 'in_mutpb': 0.30000000000000004, 'pop_size': 50, 'n_offspring': 3, 'tournsize_parent': 5, 'cxpb': 0.9, 'cx_type': 'blend', 'alpha': 0.9, 'mutpb': 0.9, 'mut_type': 'adaptive', 'sigma': 1.5, 'indpb': 0.2, 'tau': 0.7000000000000001, 'tournsize_survival': 5, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 356.8000000000009.


...

30 	150   	5.28    	74.2 	41.998 	249    
gen	nevals	Mean  	Max	Std    	max_hof
0  	50    	-95.42	200	188.234	200    
1  	107   	155.988	200	83.6758	200    
2  	120   	122.112	200	129.934	200    
3  	109   	151.296	200	110.484	200    
4  	103   	170.744	200	91.0885	200    
5  	115   	153.08 	200	110.84 	200    
6  	101   	177.128	200	81.9288	200    
7  	100   	194.748	200	26.5575	200    
8  	103   	172.88 	200	87.8053	200    
9  	113   	127.136	200	119.384	200    
10 	117   	128.656	200	132.706	200    
11 	96    	187.8  	200	57.3253	200    
12 	108   	143.784	200	125.588	200    
13 	108   	160.584	200	100.823	200    
14 	108   	159.104	200	94.8673	200    
15 	103   	163.528	200	95.2545	200    
16 	115   	136.248	200	119.774	200    
17 	102   	190.744	200	45.3535	200    
18 	109   	138.168	200	124.588	200    
19 	99    	191.924	200	56.532 	200    
20 	111   	95.524 	200	156.589	200    
21 	111   	142.46 	200	119.752	200    
22 	104   	125.792	200	131.269	200    
23 	107   	182.564	200	6

[I 2023-10-15 19:11:27,423] Trial 13 finished with value: 200.00000000000088 and parameters: {'in_sigma': 0.30000000000000004, 'in_tau': 1.0, 'in_mutpb': 0.8, 'pop_size': 50, 'n_offspring': 3, 'tournsize_parent': 10, 'cxpb': 0.1, 'cx_type': 'whole_aritmetic', 'alpha': 'random', 'mutpb': 0.7000000000000001, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 0.1, 'tau': 1.0, 'tournsize_survival': 5, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 356.8000000000009.


30 	117   	111.44 	200	144.627	200    
gen	nevals	Mean  	Max	Std    	max_hof
0  	50    	90.452	249	135.966	249    
1  	200   	-290.668	62.4	164.668	249    
2  	200   	-233.328	68  	251.03 	249    
3  	200   	-124.772	79.6	136.759	249    
4  	200   	-120.1  	93  	134.696	249    
5  	200   	-59.356 	39.8	69.5548	249    


Process ForkPoolWorker-103:
Process ForkPoolWorker-101:
Process ForkPoolWorker-102:
Process ForkPoolWorker-104:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 3

KeyboardInterrupt: 

In [ ]:
#Trial 16 finished with value: 30.800000000000495 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 6, 'cxpb': 0.6, 'cx_type': 'blend', 'alpha': 0.8, 'mutpb': 0.9, 'mut_type': 'adaptive', 'sigma': 3.0, 'indpb': 0.8, 'tau': 0.30000000000000004, 'tournsize_survival': 7, 'ngen': 30, 'verbose': 1}

In [ ]:
#Trial 0 finished with value: 356.8000000000009 and parameters: {'in_sigma': 0.7000000000000001, 'in_tau': 0.9, 'in_mutpb': 0.6, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 1, 'cxpb': 0.30000000000000004, 'cx_type': 'whole_aritmetic', 'alpha': 'random', 'mutpb': 0.9, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 0.1, 'tau': 0.8, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}

In [22]:
params = {'in_sigma': 0.7,
          'in_tau': 0.9,
          'in_mutpb': 0.6,
          'pop_size': 50,
          'n_offspring': 4,
          'tournsize_parent': 1,
          'cxpb': 0.3,
          'cx_type': 'whole_aritmetic',
          'alpha': 'random',
          'mutpb': 0.9,
          'mut_type': 'normal',
          'sigma': 2.5,
          'indpb': 0.1,
          'tau': 0.8,
          'tournsize_survival': 10,
          'ngen': 100,
          'verbose': 1}

In [23]:
logs_list = []
for i in range(100):
    hof,  logs = genetic_algorithm(**params)
    logs_list.append(hof)

gen	nevals	Mean   	Max  	Std    	max_hof
0  	50    	-71.104	263.4	269.315	263.4  
1  	180   	55.276 	263.4	93.3687	263.4  
2  	184   	96.52  	259.2	86.4545	263.4  
3  	185   	103.812	259.2	75.202 	263.4  
4  	180   	136.548	259.2	68.3582	263.4  
5  	185   	118.708	259.2	49.8873	263.4  
6  	195   	124.612	236.4	42.4792	263.4  
7  	185   	145.996	188.6	35.6654	263.4  
8  	184   	153.596	196  	34.1448	263.4  
9  	185   	156.088	196  	26.0572	263.4  
10 	186   	169.872	206  	20.3634	263.4  
11 	190   	159.576	206  	34.813 	263.4  
12 	185   	169.38 	216.2	24.8179	263.4  
13 	194   	161.76 	205.8	36.8038	263.4  
14 	185   	177.92 	217.4	22.2043	263.4  
15 	187   	173.084	217.4	32.5707	263.4  
16 	187   	176.436	211.4	23.8607	263.4  
17 	185   	177.428	221.6	31.4143	263.4  
18 	188   	175.004	221.6	34.0706	263.4  
19 	185   	173.72 	212.2	26.9334	263.4  
20 	187   	177.32 	210.6	26.154 	263.4  
21 	180   	184.572	220  	28.3144	263.4  
22 	180   	186.512	216.4	22.2311	263.4  
23 	188   	187.1

Process ForkPoolWorker-44:
Process ForkPoolWorker-42:
Process ForkPoolWorker-43:
Process ForkPoolWorker-41:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._ta

KeyboardInterrupt: 

In [31]:
son = pd.DataFrame(logs_list)
son['results'] = pd.DataFrame(logs_list)['best_solution'].apply(lambda x: final_res(np.array(x)))

In [40]:
yaz = son.loc[1,'best_solution']

In [44]:
son.loc[1,'results']

[(94.78506424239102, 100, 0, 184),
 (92.44704041507838, 80, 0, 258),
 (89.44935744741228, 54, 0, 384),
 (85.86085271205962, 23.800000000000246, 0, 678),
 (92.18901301948915, 77.80000000000021, 0, 268),
 (3.2131026186332923, 0, 90.0, 326),
 (92.93833644349735, 80.80000000000018, 0, 171),
 (90.39794861721987, 61.600000000000335, 0, 318)]

In [41]:
with open('best.txt','w') as file:
    for item in yaz:
        file.write((f"{item}\n"))

In [37]:
son.loc[3,'results']

[(94.78506424239102, 100, 0, 184),
 (92.49061166337202, 80, 0, 247),
 (88.61585110906252, 48, 0, 485),
 (84.36436000992569, 11.80000000000023, 0, 912),
 (92.18901301948915, 77.80000000000021, 0, 268),
 (3.30964054567594, 0, 90.0, 296),
 (92.93833644349735, 80.80000000000018, 0, 171),
 (91.7666105112725, 73.00000000000026, 0, 253)]

In [26]:
def final_res(sol):
    global env
    res = []
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res.append(env.play(sol))
    return res

In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""